In [3]:
import numpy as np
import cv2
import math
import requests
import pandas as pd
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm
import urllib.parse as parse
import os
import json
import pytubefix
from pytubefix import YouTube
from pytubefix.exceptions import VideoUnavailable, VideoPrivate
from Katna.video import Video
from Katna.writer import KeyFrameDiskWriter
from transformers import AutoProcessor, AutoModelForCausalLM  
from PIL import Image
import requests
import copy
import supervision as sv
from IPython.display import display
import re

In [4]:
# point to videoxum train file 
train_file = "./dataset/train_videoxum.json"

In [5]:
with open(train_file, 'r') as file:
    train_data = json.load(file)

In [19]:
if not os.path.isdir("./dataset/video_folder"): 
    os.mkdir("./dataset/video_folder")
if not os.path.isdir("./dataset/keyframes"):
    os.mkdir("./dataset/keyframes")

In [25]:
video_id_list = [] 
description_gt_list = [] 
youtube_urls = []

for i in range(1): 
    datapoint = train_data[i] 
    video_id = datapoint["video_id"] 
    print(i+1, video_id)
    video_file = video_id.split("_")[-1]
    youtube_file = "https://www.youtube.com/watch?v="+video_file

    # download location for .mp4 files (change dir) 
    file_dir = "./dataset/video_folder/" 
    file_path = os.path.join(file_dir, video_file+".mp4")
    try: 
        yt = YouTube(youtube_file) 
        stream = yt.streams.get_highest_resolution()
        stream.download(output_path="./dataset/video_folder/", filename=video_file+".mp4")
        youtube_urls.append(youtube_file) 
        video_id_list.append(video_file)
        description_gt_list.append(" ".join(datapoint["tsum"]))
    except VideoUnavailable:
        print("The video is unavailable.")
    except VideoPrivate:
        print("The video is private.")
    except Exception as e:
        print(f"An error occurred: {e}")
    

1 v_QOlSCBRmfWY


In [27]:
data = {"video_id": video_id_list, 
        "description": description_gt_list, 
        "youtube_url": youtube_urls}
df = pd.DataFrame(data)

In [177]:
for i in range(len(video_id_list)): 
    
    video_file = video_id_list[i]
    print("Video "+str(i)+": ", video_file, end = " ")

    # point to .mp4 folder
    file_path = os.path.join("./dataset/video_folder", video_file+".mp4")

    # create new folder to store keyframes
    keyframe_location = os.path.join("./dataset/keyframes", video_file)
    
    if not os.path.exists("./dataset/keyframes"): 
        os.mkdir("./dataset/keyframes") 
    if not os.path.exists(keyframe_location):
        os.mkdir(keyframe_location)

    vd = Video()

    video = cv2.VideoCapture(file_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
    if total_frames == 0 or fps == 0: 
        continue
    duration = math.floor(total_frames / fps)
    
    # Print the results
    # print(f"Frames per second: {fps}")
    # print(f"Total number of frames: {total_frames}")
    # print(f"Video duration (seconds): {duration}")
    
    no_of_frames_to_returned = min(int(duration/10), 15)
    print("number of keyframes = ", no_of_frames_to_returned, "total frames = ", total_frames, "video duration = ", duration)
    
    diskwriter = KeyFrameDiskWriter(location=keyframe_location)
    
    vd.extract_video_keyframes(
       no_of_frames=no_of_frames_to_returned, file_path=file_path,
       writer=diskwriter
    )

    os.remove(file_path)



Video 0:  dAjhBWalWRM number of keyframes =  3 total frames =  961.0 video duration =  32


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Completed processing for :  /Users/arpitasahu/Desktop/USC/2024-25/Fall'24/CSCI 544 (Applied NLP)/project/dataset/video_folder/dAjhBWalWRM.mp4
Video 1:  zihMznAYlV0 number of keyframes =  7 total frames =  2179.0 video duration =  72


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Completed processing for :  /Users/arpitasahu/Desktop/USC/2024-25/Fall'24/CSCI 544 (Applied NLP)/project/dataset/video_folder/zihMznAYlV0.mp4
Video 2:  epqnpUCVLK8 number of keyframes =  6 total frames =  1516.0 video duration =  63


...

Completed processing for :  /Users/arpitasahu/Desktop/USC/2024-25/Fall'24/CSCI 544 (Applied NLP)/project/dataset/video_folder/0SmA1cGocz4.mp4
Video 110:  aDDq5ofaFHI number of keyframes =  15 total frames =  3763.0 video duration =  156


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Completed processing for :  /Users/arpitasahu/Desktop/USC/2024-25/Fall'24/CSCI 544 (Applied NLP)/project/dataset/video_folder/aDDq5ofaFHI.mp4
Video 111:  4d3RtnuVCV4 number of keyframes =  12 total frames =  3038.0 video duration =  126


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Completed processing for :  /Users/arpitasahu/Desktop/USC/2024-25/Fall'24/CSCI 544 (Applied NLP)/project/dataset/video_folder/4d3RtnuVCV4.mp4


In [17]:
# code block for single line key frame description (using vit gpt) 

# device = "cuda" if torch.cuda.is_available() else "cpu"
# finetuned_model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning").to(device)
# finetuned_tokenizer = GPT2TokenizerFast.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# finetuned_image_processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# def load_image(image_path):
#     if os.path.exists(image_path):
#         return Image.open(image_path)
#     else: 
#         print("image does not exist") 
#         return None
        
# # a function to perform inference
# def get_caption(model, image_processor, tokenizer, image_path):
#     image = load_image(image_path)
#     if image is None: 
#         return None 
#     # preprocess the image
#     img = image_processor(image, return_tensors="pt").to(device)
#     # generate the caption (using greedy decoding by default)
#     output = model.generate(**img)
#     # decode the output
#     caption = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
#     return caption
    
# caption_dict = {}
# dir = "/Users/arpitasahu/Desktop/USC/2024-25/Fall'24/CSCI 544 (Applied NLP)/project/dataset/keyframes" 
# for folder in os.listdir(dir): 
#     print("video = ", folder) 
#     folder_dir = os.path.join(dir, folder)
#     if not os.path.isdir(folder_dir): 
#         print("skipping") 
#         continue
#     caption_dict[folder] = ""
#     captions = ""
#     for file in os.listdir(folder_dir): 
#         if file == '.DS_Store':
#             print("skipping ds store") 
#             continue
#         file = os.path.join(folder_dir, file)
#         # display(load_image(file))
#         # get the caption
#         caption = get_caption(finetuned_model, finetuned_image_processor, finetuned_tokenizer, file)
#         # print(caption)
#         if caption is not None: 
#             caption_dict[folder] += caption
#             caption_dict[folder]+=". "
#     # caption_list.append(captions)

# df['vit_keyframe_descriptions'] = df['video_id'].map(caption_dict)

In [178]:
# dense captioning using florence-2-large 

model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

def run_example(task_prompt, image, text_input=None):
    if text_input is None:
       	prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    generated_ids = model.generate(
      input_ids=inputs["input_ids"],
      pixel_values=inputs["pixel_values"],
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
     generated_text,
     task=task_prompt,
     image_size=(image.width, image.height)
    )

    return parsed_answer



loading configuration file config.json from cache at /Users/arpitasahu/.cache/huggingface/hub/models--microsoft--Florence-2-large/snapshots/f0acedbf9b780e04fe1f9111fcf53187388f3d03/config.json
loading configuration file config.json from cache at /Users/arpitasahu/.cache/huggingface/hub/models--microsoft--Florence-2-large/snapshots/f0acedbf9b780e04fe1f9111fcf53187388f3d03/config.json
Model config Florence2Config {
  "_name_or_path": "microsoft/Florence-2-large",
  "architectures": [
    "Florence2ForConditionalGeneration"
  ],
  "auto_map": {
    "AutoConfig": "microsoft/Florence-2-large--configuration_florence2.Florence2Config",
    "AutoModelForCausalLM": "microsoft/Florence-2-large--modeling_florence2.Florence2ForConditionalGeneration"
  },
  "bos_token_id": 0,
  "eos_token_id": 2,
  "ignore_index": -100,
  "is_encoder_decoder": true,
  "model_type": "florence2",
  "pad_token_id": 1,
  "projection_dim": 1024,
  "text_config": {
    "_attn_implementation_autoset": false,
    "_name_or

In [179]:
video_caption_dict = {}

In [ ]:
import re

task_prompt = "<MORE_DETAILED_CAPTION>"

# point to keyframes folder 
vid_folder = "/Users/arpitasahu/Desktop/USC/2024-25/Fall'24/CSCI 544 (Applied NLP)/project/dataset/keyframes" 

for frame_folder in os.listdir(vid_folder): 
    caption_list = []
    if frame_folder == '.DS_Store': 
        continue
    print("video id = ", frame_folder)
    frame_folder_path = os.path.join(vid_folder, frame_folder) 
    for file in os.listdir(frame_folder_path): 
        if file == '.DS_Store':
            continue 
        print("frame = ", file)
        frame_file = os.path.join(frame_folder_path, file)
        image = Image.open(frame_file) 
        # image = image.convert("RGB")
        # display(ima"ge)
        answer = run_example(task_prompt=task_prompt, image = image)
        caption =  answer['<MORE_DETAILED_CAPTION>']
        if caption.startswith("The image shows "): 
            caption = re.sub("The image shows ", "", caption)
        # print("description = ",caption)
        caption_list.append(caption)
    video_caption_dict[frame_folder] = caption_list

video id =  5lre5zbUV1Q
frame =  5lre5zbUV1Q_1.jpeg


Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 3,
  "pad_token_id": 1
}



frame =  5lre5zbUV1Q_0.jpeg


Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 3,
  "pad_token_id": 1
}



frame =  5lre5zbUV1Q_6.jpeg


...

frame =  ffyPN2-EDg4_1.jpeg


Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 3,
  "pad_token_id": 1
}



frame =  ffyPN2-EDg4_0.jpeg


Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 3,
  "pad_token_id": 1
}



frame =  ffyPN2-EDg4_3.jpeg


In [ ]:
df['dense_frame_descriptions'] = df['video_id'].map(video_caption_dict)

In [ ]:
# df = df.drop('bart_dense_descriptions', axis=1)

In [ ]:
df = df[df['dense_frame_descriptions'].apply(lambda x: len(x) > 0)]

In [ ]:
df

In [ ]:

# df.to_csv('/Users/arpitasahu/Desktop/USC/2024-25/Fall\'24/CSCI 544 (Applied NLP)/project/dense_descriptions.csv', index=False) 

In [103]:
# video_caption_dict

{'cp52LdlmlUk': ['a group of people in a gymnasium with a blue floor and white walls. There are six people in the image, three women and three men, all wearing workout clothes. They are standing on black treadmills and appear to be in the middle of a workout routine. The woman in the center of the image is wearing a blue tank top and black pants and is holding a black dumbbell in her hand. The other three women are wearing yellow tank tops and black leggings. There is a large screen in the background with a colorful design on it. The image appears to be taken from a high angle, looking down on the group.',
  'a group of people in a gymnasium. There are four people in the image, three women and one man, all wearing workout clothes. The woman in the center is wearing a blue tank top and black leggings, while the other two women are wearing black tank tops and leggins. They are all holding dumbbells and appear to be in the middle of a workout routine.\n\nIn the background, there is a larg

In [ ]:
# text summarization using bart 

from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


In [ ]:
summary_dict = {} 

for key, val in video_caption_dict.items():
    print("video = ", key) 
    document = " ".join(val) 
    tokens = tokenizer(document, max_length=1024, truncation=True, return_tensors="pt")
    truncated_text = tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)
    summary = summarizer(truncated_text, max_length=200, min_length=150, do_sample=False)
    
    summary_output = summary[0]['summary_text']
    # print("Generated Paragraph:", summary_output)
    summary_dict[key] = summary_output

In [ ]:
df['bart_summarized_video_captions'] = df['video_id'].map(summary_dict)

In [ ]:
# change CSV save location
df.to_csv('/Users/arpitasahu/Desktop/USC/2024-25/Fall\'24/CSCI 544 (Applied NLP)/project/summarized_dense_descriptions_100.csv', index=False) 

In [ ]:
df

In [37]:
summary_output

'Group of people standing on top of a basketball court, with a wall in the background adorned with photo frames and windows on the right side. People appear to be in the middle of a zumba class, with some of them holding objects in their hands. The image is slightly blurred, giving it a dream-like quality. a group of people in a gym with a. wall in a background, a door on the left side, and a few objects on the floor. The people are all wearing different colored dresses and are using steppers to move around the gym. A group of. people riding skateboards on top. of abasketball court. The images are slightly blurred to give them a more realistic look, with objects scattered around the court.'

In [ ]:
# Summarization using T5 large 

# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# model_name = "t5-large"  # You can also try "facebook/bart-large-cnn" or "t5-large"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# for key, val in video_caption_dict.items():
#     input_text = " ".join(val)
#     inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=2000, truncation=True)
    
#     # Generate the rephrased text
#     outputs = model.generate(
#         inputs,
#         max_length=150,
#         num_beams=5,      # Beam search for better results
#         temperature=1.0,  # Adjust for diversity
#         repetition_penalty=1.2,  # Penalize repeated phrases
#     )
    
#     # Decode and print the output
#     rephrased_paragraph = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print("Rephrased Paragraph:")
#     print(rephrased_paragraph)

In [85]:
# from rouge import Rouge
# rouge_scorer = Rouge()

# hypothesis = summary_output
# reference = gt

# score = rouge_scorer.get_scores(
#     hyps=hypothesis,
#     refs=reference,
# )
# print(score[0])

{'rouge-1': {'r': 0.5, 'p': 0.1232876712328767, 'f': 0.19780219462866805}, 'rouge-2': {'r': 0.21739130434782608, 'p': 0.041666666666666664, 'f': 0.06993006723067152}, 'rouge-l': {'r': 0.5, 'p': 0.1232876712328767, 'f': 0.19780219462866805}}
